In [10]:
'''Dada uma sequencia e um trecho de interesse, encontrar os hits na sequencia e 
 e calcular as distancias entre os hits e o trecho de interesse'''
import bisect
   
class SubseqIndex(object):
    
    TOTAL_SUBSEQS = 2
    SUBSEQS = {0: 'first', 1:'second'}

    def __init__(self, t, k, ival):
        """ival: intervalo entre os caracteres do texto 
           k: quantidade de caracteres na chave do indice
           t: texto
        Transforma um texto em um indice, cujos elementos sao subsequencias do tamanho (k)
        e extraidas de forma sequencial, respeitando o espaco  definido  entre os caracteres (ival).
        Permite apenas realizar buscas exatas.
        Ex: SubseqIndex("ATAT", 2, 2)  retorna indice ("AA", 0) and ("TT", 1)"""
        self.k = k  # num characters per subsequence 
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)

        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        '''p: padrao buscado
        Obtem os indices encontrados para determinada busca. Para isso,
        extrai subsequencia do trecho buscado considerando o mesmo padrao que foi usado 
        para a montagem do indice. Em seguida, faz a busca no indice. Se nao encontrar, 
        faz uma nova tentativa, porem começando a subsequencia do segundo caracter. 
        Retorna os hits e se foi na primeira ou na segunda subsequencia '''
        hits = []

        for i in range (self.TOTAL_SUBSEQS):
            
            subseq = p[i:i+self.span:self.ival]  # query with  subseq
            print('subseq ', self.SUBSEQS[i], ':' , subseq)
            
            j = bisect.bisect_left(self.index, (subseq, -1))  # binary search
            
            while j < len(self.index):  # collect matching index entries
                if self.index[j][0] != subseq:
                    break
                hits.append([self.index[j][1], self.SUBSEQS[i]]) #[pos, first or second subseq]
                j += 1
                            
            if  len(hits) > 0:
                break
                
        print('hits',len(hits))
        return hits


In [11]:
def approximate_edit_distance(x, texto, start):
    # Create distance matrix
    D = []
    
    #extrai o trecho do texto que corresponde ao trecho do match 
    y = texto[start:start+len(x)]
    
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]


In [12]:
def leitura_fasta(nome_arquivo):
    'Processamento de arquivo fasta e retorno de dicionario com as sequencias lidas'
    
    sequencias = {}
    seq_id = ''
    
    try:
        arquivo_fasta =  open(nome_arquivo)
    except 'IOError':
       print('Arquivo nao encontrado!') 

    for linha in arquivo_fasta:
        if linha[0] == '>':
            seq_id = linha.rstrip()[1:linha.find(' ')]
            sequencias[seq_id] = ''
        elif linha != '':
            sequencias[seq_id] = sequencias[seq_id] + linha.rstrip()
    print('quantidade de sequencias lidas=>', len(sequencias))
    return sequencias

In [13]:
#obtem o arquivo fasta
arquivo = 'chr1.GRCh38.excerpt.fasta'
sequencias = leitura_fasta(arquivo)
size = 2
span = 1

#monta um indice com os dados da sequencia onde começam com a letra 
index = SubseqIndex(sequencias['CM000663.2_excerpt'], size, span)

quantidade de sequencias lidas=> 1


In [14]:
#busca as entradas no indice (hits) que correspondem aos trechos buscados
trecho1 = 'GCTGATCGATCGTACG'
trecho2 = 'GATTTACCAGATTGAG'
hits1 = index.query(trecho1)
hits2 = index.query(trecho2)

subseq  first : GC
hits 26485
subseq  first : GA
hits 46202


In [15]:
#para cada hit no indice, calcula a distancia e guarda o resultado
distancias_trecho_1 = {}
for hit in hits1:
    distancia = approximate_edit_distance(trecho1, sequencias['CM000663.2_excerpt'], hit[0])
    distancias_trecho_1[hit[0]] = distancia
distancias_trecho_1_sorted =  sorted(distancias_trecho_1.items(), key=lambda x: x[1])

In [16]:
#para cada hit no indice, calcula a distancia e guarda o resultado
distancias_trecho_2 = {}
for hit in hits2:
    distancia = approximate_edit_distance(trecho2, sequencias['CM000663.2_excerpt'], hit[0])
    distancias_trecho_2[hit[0]] = distancia
distancias_trecho_2_sorted =  sorted(distancias_trecho_2.items(), key=lambda x: x[1])

In [17]:
#Menor distancia do trecho 1:
trecho_seq_1 = sequencias['CM000663.2_excerpt'][distancias_trecho_1_sorted[0][0]:distancias_trecho_1_sorted[0][0]+len(trecho1)]
print('seq:', trecho_seq_1, 'pos:', distancias_trecho_1_sorted[0][0], 'dis:', distancias_trecho_1_sorted[0][1])

seq: GCTGATGGATAGTAAG pos: 380536 dis: 3


In [18]:
#Menor distancia do trecho 2:
trecho_seq_2 = sequencias['CM000663.2_excerpt'][distancias_trecho_2_sorted[0][0]:distancias_trecho_2_sorted[0][0]+len(trecho2)]
print('seq:', trecho_seq_2, 'pos:', distancias_trecho_2_sorted[0][0], 'dis:', distancias_trecho_2_sorted[0][1])

seq: GATTTACCAAGATTGA pos: 421071 dis: 2
